# Using QuantumGraph

The `quantumgraph` package is designed to help build quantum software using an approach that is different from the one you'll learn in textbooks. Here we'll look into the basics of how it works.

## Setting up

The first step is to install `quantumgraph`. You'll also need to install the `pairwise_tomography` package, which is a vital dependency.

This can be done by pip installing directly from the repositories. This is usually done with the command line, but you can do it in Jupyter notebooks too. The syntax for Jupyter notebooks is as follows.

!pip install git+https://github.com/moth-quantum/QuantumGraph.git

With these things installed, you will need to import the class in which all the magic happens.

In [3]:
from quantumgraph import QuantumGraph

## Initializing the variables

Computing is all about encoding information in variables, and then manipulating these variables. So to start we need to understand what variables we will use and how to set them up.

The variables in `QuantumGraph` are associated with a set of qubits. We begin by creating a `QuantumGraph` object to hold our qubits. In the example below, we do it for a set of four qubits.

In [4]:
n = 4
qubits = QuantumGraph(n)

For each qubit we have three variables, $\langle X \rangle$ , $\langle Y \rangle$ and $\langle Z \rangle$. These are all numbers in the range between $+1$ and $-1$, and they satisfy

$$
\langle X \rangle^2 + \langle Y \rangle^2 + \langle Z \rangle^2 \leq 1.
$$

These values for a qubit `j` can be accessed as a dictionary using the `get_bloch()` method. Here it is in use.

In [5]:
for j in range(n):
    print('Single qubit variables for qubit ' + str(j) + ':')
    print(qubits.get_bloch(j))
    print()

Single qubit variables for qubit 0:
{'X': np.float64(0.0024898891182221763), 'Y': np.float64(0.003071536545418576), 'Z': np.float64(0.995020137423067)}

Single qubit variables for qubit 1:
{'X': np.float64(0.0026443331278610408), 'Y': np.float64(0.00030209370520759276), 'Z': np.float64(0.9943038315239268)}

Single qubit variables for qubit 2:
{'X': np.float64(0.003489222783690582), 'Y': np.float64(0.0026400848786104214), 'Z': np.float64(0.9924766779432712)}

Single qubit variables for qubit 3:
{'X': np.float64(0.004142204533513063), 'Y': np.float64(0.010381291955312915), 'Z': np.float64(0.9937894256632561)}



For each possible pair of qubits $j$ and $k$, we have a whole bunch of variables: $\langle X_j X_k \rangle$, $\langle X_j Y_k \rangle$, $\langle X_j Z_k \rangle$, $\langle Y_j X_k \rangle$, $\langle Y_j Y_k \rangle$, etc. For a fairly in-depth explation of these, see [this paper](https://arxiv.org/abs/2005.10327).

For now it is sufficient to note these variables are initially completely dependent on the single qubit variables. For example

$$
\langle X_j Y_k \rangle = \langle X_j \rangle \langle Y_k \rangle.
$$

They can begin to do different things as a computation proceeds, but they'll always satisfy

$$
\langle X_j Y_k \rangle^2 \geq \left( \langle X_j \rangle \langle Y_k \rangle \right)^2
$$

These values for a pair of qubits `j` and `k` can be similarly accessed using the `get_relationship()` method.

In [6]:
for j in range(n-1):
    for k in range(j+1,n):
        print('Two qubit variables for qubits ' + str(j) + ' and '  + str(k) + ':')
        print(qubits.get_relationship(j,k))
        print()

Two qubit variables for qubits 0 and 1:
{'XX': np.float64(0.0018013064306953387), 'XY': np.float64(0.003387747501115904), 'XZ': np.float64(0.004245307813172669), 'YX': np.float64(0.004235515283057151), 'YY': np.float64(0.002750602607777611), 'YZ': np.float64(0.0031762191098927864), 'ZX': np.float64(0.003399912502911465), 'ZY': np.float64(0.000752685177833649), 'ZZ': np.float64(0.9916026346658723)}

Two qubit variables for qubits 0 and 2:
{'XX': np.float64(0.002140762492272935), 'XY': np.float64(-0.012650700418070421), 'XZ': np.float64(0.0035684906590037944), 'YX': np.float64(-0.011764513318394657), 'YY': np.float64(-0.004506973385874821), 'YZ': np.float64(0.0025028942947366833), 'ZX': np.float64(0.0021318540449456053), 'ZY': np.float64(0.0031835869937233965), 'ZZ': np.float64(0.9912426404531735)}

Two qubit variables for qubits 0 and 3:
{'XX': np.float64(-0.0014857422699779397), 'XY': np.float64(0.004388527394674209), 'XZ': np.float64(0.001987061252687379), 'YX': np.float64(0.002809758

The results we see above show us that we are in the standard $|000\rangle$ initial state, for which

$$\langle Z_j \rangle = \langle Z_j Z_k \rangle = 1 \,\,\,\, \forall j,k$$

All variables involving $X$ and $Y$ are zero. Any deviations from this seen above are due to the fact that the expectation values are statistically calculated.

## Manipulating qubits

The qubits can be manipulated using normal quantum gates. For this we can directly manipulate the quantum circuit for the qubits, which can be accessed through the attribute `qc`. Since `QuantumGraph` is built on Qiskit, this circuit is a Qiskit `QuantumCircuit` object.

In [7]:
qubits.qc.h(0)
qubits.qc.draw()

┌───┐
q_0: ┤ H ├
     └───┘
q_1: ─────
          
q_2: ─────
          
q_3: ─────

Here we applied an `h` gate to qubit 0. However, if we look at the state of this qubit we will find that it has not changed.

In [8]:
qubits.get_bloch(0)

{'X': np.float64(0.0024898891182221763),
 'Y': np.float64(0.003071536545418576),
 'Z': np.float64(0.995020137423067)}

This is because we actually have to run the circuit in order to update the tomography. This is done using the `update_tomography()` method.

In [9]:
qubits.update_tomography()
print(qubits.get_bloch(0))

{'X': np.float64(0.9952428831453248), 'Y': np.float64(0.005790050589214698), 'Z': np.float64(-0.00480181995404992)}


Now we have $\langle X \rangle = 1$ for qubit 0. If you know what an `h` does, this shouldn't come as a surprise (if you didn't know, then now do!).

Another way to manipulate the qubits, which is unique to this method, is to give target values for the variables. This is done using the method `set_bloch()`.

For example, to set the state of qubit 1 to $\langle X \rangle = 1$ we use

In [10]:
qubits.set_bloch({'X':1},1)

This automatically updates the tomography (unless the keyword `update=False` is added), so we can already verify that it has the correct effect.

In [11]:
print(qubits.get_bloch(1))

{'X': np.float64(0.9932071843427954), 'Y': np.float64(0.0005402271205398258), 'Z': np.float64(-0.003368384175604422)}


The effect was acheived by considering the initial and target tomography, and finding a unitary to get as close to the target as possible.

In [12]:
qubits.qc.draw()

┌───┐               
q_0: ───────────────┤ H ├───────────────
     ┌──────────────┴───┴──────────────┐
q_1: ┤ U(1.5699,-0.00024923,1.066e-07) ├
     └─────────────────────────────────┘
q_2: ───────────────────────────────────
                                        
q_3: ───────────────────────────────────

It is not as pretty as the off-the-shelf Hadamard gate, but it has the same effect.

The target state for the `set_bloch()` method can include multiple values. For example


In [13]:
qubits.set_bloch({'X':1, 'Z':1},2)

print(qubits.get_bloch(2))

{'X': np.float64(0.7015359409335796), 'Y': np.float64(-0.007762748415110434), 'Z': np.float64(0.7034343701922496)}


Note that it is not actually possible to set both $\langle X \rangle$ and $\langle Z \rangle$ of a qubit to $1$, so `set_bloch` just does the best that is possible. For all single qubit values that do not appear in the target state, it is assumed that the target is to set them to zero. So the target used here is equivalent to `{'X':1, 'Y':0, 'Z':1,}`.


We can perform similar manipulations on pairs of qubits using the `set_relationship` method. Here we do so for qubits 0 and 1.

In [14]:
qubits.set_relationship( {'ZZ':1}, 0,1 )

print(qubits.get_relationship(0,1))

{'XX': np.float64(0.9907612683977022), 'XY': np.float64(-0.0006892142658266162), 'XZ': np.float64(0.007465906386810411), 'YX': np.float64(0.0013805060760822255), 'YY': np.float64(-0.9913504482520893), 'YZ': np.float64(0.00245416284092698), 'ZX': np.float64(-0.008264406159421048), 'ZY': np.float64(0.0010266817318537115), 'ZZ': np.float64(0.991453504836959)}


Unlike `set_bloch`, it is not assumed that values not present in the target should be set to zero. Instead, it is assumed that they should be changed as little as possible.

In the example above, the target is to set $\langle Z_0 Z_1 \rangle = 1$. Note that the state already has $\langle X_0 \rangle = \langle X_0 \rangle = 1$, and therefore $\langle X_0 X_1 \rangle = 1$, from the single qubit manipulations we have already performed. Since $Z_0 Z_1$ and $X_0 X_1$ commute, it is possible to satisfy the target condition on $\langle Z_0 Z_1 \rangle$ while maintaining the value of $\langle X_0 X_1 \rangle$. It is because of this that we end up with a state with $\langle X_0 X_1 \rangle = \langle Z_0 Z_1 \rangle = 1$.

We can also use `set_relationship()` with multiple values in the target. However, unlike for `set_bloch()`, all the corresponding Paulis must commute to get a well defined effect. For example, for qubits 2 and 3.

In [15]:
qubits.set_relationship( {'XZ':1,'ZX':1}, 2,3 )

print(qubits.get_relationship(2,3))

{'XX': np.float64(0.0038122386214630042), 'XY': np.float64(0.0016067900944804338), 'XZ': np.float64(0.9915280336469519), 'YX': np.float64(0.0020669714987316635), 'YY': np.float64(0.9927802783736336), 'YZ': np.float64(-0.0013609942490333227), 'ZX': np.float64(0.9935332920459953), 'ZY': np.float64(-0.0023511390862160624), 'ZZ': np.float64(-0.0011435615859736988)}


Both `set_bloch` and `set_relationship` also have a `fraction` keyword argument. If we use $U$ to denote the unitary that would have been applied normally, setting a value for `fraction` applies $U^{\mathtt{fraction}}$ instead.

Let's get a fresh pair of qubits to see an example of this.

In [16]:
qubits = QuantumGraph(2)

print(qubits.get_relationship(0,1))

{'XX': np.float64(-0.0042972942487531605), 'XY': np.float64(-0.0040597136306738), 'XZ': np.float64(0.007392323988747366), 'YX': np.float64(0.007252521238175376), 'YY': np.float64(0.006594896611599482), 'YZ': np.float64(-0.008996043125424279), 'ZX': np.float64(0.009769100521249305), 'ZY': np.float64(-0.01086920376509145), 'ZZ': np.float64(0.9883046566216618)}


Applying `qubits.set_relationship({'XX':1}, 0,1)` would change the state to get $\langle X_0 X_1 \rangle = 1$. But let's add in `fraction=0.5` and see what happens.

In [17]:
qubits.set_relationship( {'XX':1}, 0,1, fraction=0.5 )

print(qubits.get_relationship(0,1))

{'XX': np.float64(0.5727632573744366), 'XY': np.float64(0.27972522165762326), 'XZ': np.float64(-0.2568837681284492), 'YX': np.float64(0.2056100364870816), 'YY': np.float64(-0.6887304997036652), 'YZ': np.float64(-0.25221389290627394), 'ZX': np.float64(0.37970955715980426), 'ZY': np.float64(0.10424000410958989), 'ZZ': np.float64(0.8498411061535642)}


Here the value has increased $1/\sqrt{2}$, but not all the way to $1$. In some sense, it is half way there.

If we were to apply exactly the same quantum gate operation again, we would get all the way to $\langle X_0 X_1 \rangle = 1$. However, note that we cannot do this by simply repeating the same command, since this would generate a new gate which rotates halfway between the current initial state and the target state.

In [18]:
qubits.set_relationship( {'XX':1}, 0,1, fraction=0.5 )

print(qubits.get_relationship(0,1))

{'XX': np.float64(0.8315119318595957), 'XY': np.float64(-0.13730704511974576), 'XZ': np.float64(-0.056849478423681876), 'YX': np.float64(-0.18124981700744183), 'YY': np.float64(-0.8370389687366494), 'YZ': np.float64(-0.15442411849519322), 'ZX': np.float64(0.12321687071692068), 'ZY': np.float64(-0.06556986734654369), 'ZZ': np.float64(0.9499807459597985)}


# Creating Algorithms

By thinking of terms of these single- and two-qubit variables, and by manipulating them using the methods supplied, we can create algorithms to solve problems.

A concrete example of this can be seen in

* ["A quantum procedure for map generation", by James R. Wootton](https://arxiv.org/abs/2005.10327)

where `quantumgraph` is used to implement a rudimentary AI for a Civilization-like game.

Another example is

* ["Determining eigenstates and thermal states on a quantum computer using quantum imaginary time evolution" by Motta, et al.](https://arxiv.org/abs/1901.07653)

which is presents a method for calculating ground states for Hamiltonians of interacting particles. It uses the same basic idea as is implemented in `quantumgraph`, but with a $p$-local tomography in general (for finite $p$) and with its own custom methods to set the variables.